In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.notebook import tqdm # Pour afficher une barre de progression

# Paramètres de l'augmentation
INPUT_CSV_PATH = "corpus.csv" # corpus initial
OUTPUT_CSV_PATH = "corpus_augmente_paraphrases.csv"
MODEL_NAME = "google/mt5-base" # modèle T5 multilingue
NUM_PARAPHRASES_PER_ARTICLE = 1 # Nombre de versions paraphrasées à générer par article initial
MAX_SENTENCES_PER_ARTICLE = 5 # Limite le nombre de phrases à paraphraser par article pour gérer la longueur
SENTENCE_SPLITTER = "." # Le caractère à utiliser pour diviser l'article en phrases

In [7]:
# Charger le modèle et le tokenizer
print(f"Chargement du modèle et du tokenizer : {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Détecter l'appareil (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modèle déplacé vers : {device}")

Chargement du modèle et du tokenizer : google/mt5-base...


/Users/catalinciomne/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Modèle déplacé vers : cpu


In [8]:
# Fonction pour générer des paraphrases
def generate_paraphrases(text, num_return_sequences=1):
    """
    Génère des paraphrases pour un texte donné.
    """
    input_text = text

    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    
    # Génération des paraphrases
    # num_beams pour la recherche en faisceau (meilleure qualité)
    # no_repeat_ngram_size pour éviter la répétition de n-grammes
    # max_length pour limiter la longueur des paraphrases
    # early_stopping=True pour arrêter la génération quand tous les beams sont terminés
    outputs = model.generate(
        **inputs,
        num_beams=5,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        max_length=128,
        early_stopping=True
    )

    paraphrases = []
    for output in outputs:
        decoded_text = tokenizer.decode(output, skip_special_tokens=True)
        # Supprimer les tokens <extra_id_X>
        import re
        cleaned_text = re.sub(r'<extra_id_\d+>', '', decoded_text).strip()
        paraphrases.append(cleaned_text)

    return paraphrases

# Charger le dataset initial
try:
    df_initial = pd.read_csv(INPUT_CSV_PATH)
    print(f"Dataset initial chargé depuis {INPUT_CSV_PATH}. Nombre d'articles : {len(df_initial)}")
except FileNotFoundError:
    print(f"Erreur : Le fichier {INPUT_CSV_PATH} n'a pas été trouvé.")
    exit()

Dataset initial chargé depuis corpus.csv. Nombre d'articles : 40


In [11]:
# Créer une liste pour stocker les articles augmentés
augmented_articles = []

# Ajouter les articles initiaux au dataset augmenté
augmented_articles.extend(df_initial.to_dict('records'))

# Procéder à l'augmentation
print(f"Début de la génération de {NUM_PARAPHRASES_PER_ARTICLE} paraphrases par article initial...")
for index, row in tqdm(df_initial.iterrows(), total=len(df_initial), desc="Augmentation des articles"):
    original_text = row['article']
    label = row['catégorie']

    # Diviser l'article en phrases
    sentences = [s.strip() for s in original_text.split(SENTENCE_SPLITTER) if s.strip()]
    
    # Limiter le nombre de phrases à paraphraser pour des raisons de performance et de cohérence
    sentences_to_paraphrase = sentences[:MAX_SENTENCES_PER_ARTICLE]

    generated_texts_for_article = []

    for _ in range(NUM_PARAPHRASES_PER_ARTICLE):
        paraphrased_sentences = []
        for sentence in sentences_to_paraphrase:
            # Générer une seule paraphrase par phrase pour éviter un mélange trop grand
            paraphrase = generate_paraphrases(sentence, num_return_sequences=1)
            if paraphrase:
                paraphrased_sentences.append(paraphrase[0])
            else:
                paraphrased_sentences.append(sentence) # Garder l'original si pas de paraphrase

        # Reconstruire l'article paraphrasé
        # Si l'article original a plus de phrases que MAX_SENTENCES_PER_ARTICLE,
        # on concatène les paraphrases avec le reste des phrases originales.
        reconstructed_article = (SENTENCE_SPLITTER + " ").join(paraphrased_sentences)
        if len(sentences) > MAX_SENTENCES_PER_ARTICLE:
            reconstructed_article += (SENTENCE_SPLITTER + " ").join(sentences[MAX_SENTENCES_PER_ARTICLE:])
        
        generated_texts_for_article.append(reconstructed_article)

    # Ajouter les articles paraphrasés à la liste
    for gen_text in generated_texts_for_article:
        augmented_articles.append({"article": gen_text, "catégorie": label})

Début de la génération de 1 paraphrases par article initial...


Augmentation des articles:   0%|          | 0/40 [00:00<?, ?it/s]

In [12]:
# Sauvegarder le dataset augmenté
df_augmented = pd.DataFrame(augmented_articles)

# Convertir les labels textuels en numériques si nécessaire (par exemple 'culture' -> 0, 'sport' -> 1)
unique_labels = df_augmented['catégorie'].unique()
label_to_id = {label: i for i, label in enumerate(unique_labels)}
df_augmented['label_id'] = df_augmented['catégorie'].map(label_to_id)

print("\n--- Aperçu du dataset augmenté ---")
print(df_augmented.head())
print(f"\nNombre total d'articles après augmentation : {len(df_augmented)}")
print(f"Labels numériques créés : {label_to_id}")

df_augmented.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8")
print(f"Dataset augmenté sauvegardé sous : {OUTPUT_CSV_PATH}")


--- Aperçu du dataset augmenté ---
                                             article catégorie  label_id
0  Festival de Cannes 2025 : Denzel Washington re...   culture         0
1  Eurovision 2025 : l’Autriche remporte le conco...   culture         0
2  Werenoi, rappeur numéro un des ventes d’albums...   culture         0
3  Flammes 2024 : Tiakola rafle la mise avec quat...   culture         0
4  « Watch You Burn », l’art brûlot de Mathias Ki...   culture         0

Nombre total d'articles après augmentation : 80
Labels numériques créés : {'culture': 0, 'sport': 1}
Dataset augmenté sauvegardé sous : corpus_augmente_paraphrases.csv
